## Load Packages

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

# for transformers, 최신버전은 에러발생
!pip install transformers==4.35.2
!pip install accelerate==0.24.1

# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.9 MB/s eta 0:00:00
     ━

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 304, done.
remote: Total 304 (delta 0), reused 0 (delta 0), pack-reused 304
Receiving objects: 100% (304/304), 57.72 MiB | 12.12 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/KoChatGPT/colossalai_ChatGPT_230319
Processing /content/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

### 경로 설정

In [ ]:
# define argment
parser = argparse.ArgumentParser()
DATA_PATH = '/content/drive/MyDrive/KoMo/Dataset/total_minus.csv'
OUTPUT_PATH = '/content/drive/MyDrive/KoMo/SFT/SFT_0521'
parser.add_argument('--data_path_1_SFT', type=str, default=DATA_PATH)
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default=OUTPUT_PATH)

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 5

print(args)

Namespace(data_path_1_SFT='/content/drive/MyDrive/KoMo/Dataset/total_minus.csv', model_name='skt/kogpt2-base-v2', max_epochs=5, train_batch_size=8, output_dir='/content/drive/MyDrive/KoMo/SFT/SFT_0521')


# Finetune SFT(koAlpaca)

In [ ]:
## test & load skt gpt2 kroean
import torch
from transformers import GPT2LMHeadModel, pipeline

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))
# ['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0])
# print(generated)


generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)
# generator(
#     ["0 : **는 게임 좋아하니\n1 :",
#     "0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 :",
#     "0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : "],
#     **generation_args
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


In [ ]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

PROMPT_DICT = {
    "prompt_input": (
        "아래는 순화해야 할 욕설, 혐오 표현이 포함된 문장과 순화된 문장이 짝을 이루는 예제입니다.\n\n"
        "욕설, 혐오 표현을 순화한 응답을 작성하세요. 이때 문장에 어울리는 이모지를 붙이세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요. 이때 문장에 어울리는 이모지를 붙이세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [ ]:
from transformers import AddedToken, AutoTokenizer, AutoModelForCausalLM


emoji_tokens = [
    "😀", "😁", "😂", "🤣", "😃", "😄", "😅", "😆", "😉", "😊", "😋", "😎", "😍", "😘", "🥰", "😗", "😙", "😚",
    "🙂", "🤗", "🤩", "🤔", "🤨", "😐", "😑", "😶", "🙄", "😏", "😣", "😥", "😮", "🤐", "😯", "😪", "😫", "🥱",
    "😴", "😌", "😛", "😜", "😝", "🤤", "😒", "😓", "😔", "😕", "🙃", "🤑", "😲", "☹", "🙁", "😖", "😞", "😟",
    "😤", "😢", "😭", "😦", "😧", "😨", "😩", "🤯", "😬", "😰", "😱", "🥵", "🥶", "😳", "🤪", "😵", "😡", "😠",
    "🤬", "😷", "🤒", "🤕", "🤢", "🤮", "🤧", "😇", "🥳", "🥺", "🤠", "🤡", "🤥", "🤫", "🤭", "🧐", "🤓", "😈",
    "👿", "👹", "👺", "💀", "👻", "👽", "👾", "🤖",


    "💌", "🕳", "💣", "💎", "🔪", "🗡", "⚔", "🛡", "🚬", "⚰", "⚱", "🏺", "🔮", "📿", "💈", "⚗", "🔭", "🔬",
    "🕯", "💡", "🔦", "🏮", "📔", "📕", "📖", "📗", "📘", "📙", "📚", "📓", "📒", "📃", "📜", "📄", "📰",
    "🗞", "📑", "🔖", "🏷", "💰", "💴", "💵", "💶", "💷", "💸", "💳", "🧾", "💹", "✉", "📧", "📨", "📩",
    "📤", "📥", "📦", "📫", "📪", "📬", "📭", "📮", "🗳", "✏", "✒", "🖋", "🖊", "🖌", "🖍", "📝", "💼",
    "📁", "📂", "🗂", "📅", "📆", "🗒", "🗓", "📇", "📈", "📉", "📊", "📋", "📌", "📍", "📎", "🖇", "📏",
    "📐", "✂", "🗃", "🗄", "🗑", "🔒", "🔓", "🔏", "🔐", "🔑", "🗝", "🔨", "🪓", "⛏", "⚒", "🛠", "🗡", "⚔",
    "🔫", "🏹", "🛡", "🔧", "🔩", "⚙", "🗜", "⚖", "🦯", "🔗", "⛓", "🧰", "🧲", "🧪", "🧫", "🧬", "🔬",
    "🔭", "📡", "💉", "💊", "🩸", "🩹", "🩺", "🚪", "🛏", "🛋", "🪑", "🚽", "🚿", "🛁", "🪒", "🧴", "🧷",
    "🧹", "🧺", "🧻", "🧼", "🪣", "🧽", "🪤", "🪒", "🔑", "🗝", "🚪", "🛌", "🛋", "🛏", "🛋", "🪑", "🚽",
    "🪣", "🛁", "🪞", "🪠", "🪤", "🪒", "🪥", "🛒", "🚬", "⚰", "⚱", "🪦", "🧿", "🪔", "🪒"
]


added_emoji_tokens = [AddedToken(emoji, rstrip=False, lstrip=False, single_word=False, normalized=False, special=True) for emoji in emoji_tokens]

tokenizer = AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,
)

tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)

tokenizer.add_tokens(added_emoji_tokens)

for token in added_emoji_tokens:
    assert tokenizer.convert_tokens_to_ids(token.content) != tokenizer.unk_token_id, f"Token {token.content} was not added correctly."
print("All tokens added successfully.")


All tokens added successfully.


In [ ]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=512,
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Embedding(51201, 768)

###SFT를 위한 데이터셋 준비

In [ ]:
data = pd.read_csv(DATA_PATH)
data.head()

,Cleaned,Emojis,Profanity
0,와 그런 사람들 되게 많다더라.,와 그런 사람들 되게 많다더라. 😮,와 그딴 씹선비들 개많음 ㅋㅋㅋ 걍 니들이나 잘하세요~
1,근데 난 대학 안 가서 주변에 아저씨 뿐이라 몰라,근데 난 대학 안 가서 주변에 아저씨 🤷‍♂️ 뿐이라 몰라,대학 씹고 니 주변에 아재들밖에 없다고? 그거 니 인생 망한거 아님? ㅋㅋㅋ
2,대학교를 가야 세상 다양한 주변 사람들 많이 사귀지 왜 안 갔어,대학교를 가야 세상 다양한 주변 사람들 많이 사귀지 🤔 왜 안 갔어,대학 안간게 뭐가 문제냐 씹선비야 ㅋㅋ 니가 대학가서 좆같은 인맥이나 쌓으면서 살아...
3,난 대학교 졸업장이 필수가 되어버린 이 사회가 슬프다,난 대학교 졸업장이 필수가 되어버린 이 사회가 슬프다 😢,대학 졸업장 없으면 개돼지 취급 받는 이 사회 참 ㅈ같네 ㅉㅉ
4,지금 고졸 무시하시는 건가요? 신고합니다,지금 고졸 😤 무시하시는 건가요? 신고합니다,지금 고졸 무시하는거 씨발? 신고 갈기노~


In [ ]:
# train_len = 10000
train_len = len(data)
data = data[:train_len]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25628 entries, 0 to 25627
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Cleaned    25628 non-null  object
 1   Emojis     25628 non-null  object
 2   Profanity  25628 non-null  object
dtypes: object(3)
memory usage: 600.8+ KB


In [ ]:
# DATA_PATH_SFT = '/content/drive/MyDrive/KoMo/Dataset/data1.csv' # 10000개로 학습
DATA_PATH_SFT = '/content/drive/MyDrive/KoMo/Dataset/total_minus.csv' # 25628개로 학습
# data = pd.read_csv(DATA_PATH_SFT)
# data.info()

In [ ]:
from typing import Optional, Dict, Sequence
from torch.utils.data import Dataset
import transformers
import pandas as pd
import copy
import torch
import logging

IGNORE_INDEX = -100  # Define IGNORE_INDEX if not defined elsewhere

class SFT_dataset(Dataset):
    '''SFT dataset by wygo'''
    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        ## format
        pattern_instruction = 'prompt'  # 순화를 지시하는 명령 (프롬프트)
        pattern_input = 'input'  # 순화시킬 데이터
        pattern_output = 'completion'  # 순화된 output

        ############################################################
        ## load dataset
        list_data_csv = pd.read_csv(data_path, encoding='utf-8')
        profanity_data = list_data_csv['Profanity']
        cleaned_data = list_data_csv['Emojis']
        if True:
            print(profanity_data[0])

        ############################################################
        ## 데이터셋 만들기, source와 target

        # 입력
        sources = []
        for example in profanity_data:
            if example != "":
                prompt = f"""
Input에 욕설, 혐오 표현이 있다면 이를 찾아 예쁜 말로 순화해줘.\n\n
Input(순화할 문장):\n{example}\n\nOutput(순화한 문장):"""
            else:
                print("Error")
            sources.append(prompt)

        # 출력
        targets = []
        for example in cleaned_data:
            targets.append(f"{example}{tokenizer.eos_token}" if isinstance(example, str) else f"{tokenizer.eos_token}")
        print("Example - sources[0]", sources[0])
        print("Example - tragets[0]", targets[0])
        print("Example - sources[-1]", sources[-1])
        print("Example - tragets[-1]", targets[-1])

        if verbose:
            idx = 0
            print(sources[idx])
            print(targets[idx])
            print("Tokenizing inputs... This may take some time...")

        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = [tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list]
        return dict(
            input_ids=input_ids,
            input_ids_lens=input_ids_lens,
        )

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

# Example usage
# Make sure to define `args`, `tokenizer`, and `data_path_1_SFT` properly in your actual implementation.
train_dataset = SFT_dataset(data_path=DATA_PATH_SFT, tokenizer=tokenizer)
eval_dataset = None  # eval은 안함
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

# Check the first sample
print("Dataset len", len(train_dataset))
print('input : %s' % train_dataset.input_ids[0])
print('output: %s' % train_dataset.labels[0])

NameError: name 'DATA_PATH_SFT' is not defined

### 5 Epoch 학습

In [ ]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

"""
# 모델 추가 학습할 때 주석 해제 후 model load
CHECKPOINT_PATH = '/content/drive/MyDrive/KoMo/SFT/0521_25628_10epoch/checkpoint-15000'   # 10 Epoch , Train data -all
model = AutoModelForCausalLM.from_pretrained(CHECKPOINT_PATH)
"""

OUTPUT_DIR = '/content/drive/MyDrive/KoMo/SFT/0521_25628_10epoch'
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=2500, # after # steps model is saved
    warmup_steps=5,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    run_name="0521_0320",
    )
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

cpu


NameError: name 'data_collator' is not defined

### SFT Inference

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/KoMo/SFT/0521_2/checkpoint-2500'   # 10 Epoch , Train data 10000
model = AutoModelForCausalLM.from_pretrained(CHECKPOINT_PATH)

model.eval()
model.to(device)

def infer(input_text: str, model, tokenizer, max_length: int = 512):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


NameError: name 'AutoModelForCausalLM' is not defined

In [ ]:
sample_list = ['에휴 씨발 물가 좆같이 올라서 먹을 수 있는게 없노', '조선족 개새끼들 한국땅에서 못살게 존나 다 내쫓아야한다이기야', '개쌉레알이노 ㅋㅋㅋㅋ 웹툰보이가 그린웹툰들 몰입력 ㅆㅎㅌㅊ에 내용 ㅆㅎㅌㅊ 반면 웹툰작가꺼는 나 ㅆㅅㅌㅊ에 잼임', 'ㅋㅋㅋㅋ 상상속에서 맘충이랑 싸움했ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ일베에서나 키보드로 욕이나 하지 현실에선 말한마디나 제대로 하노?', '요즘 유흥에도 베트남년들 개많더라 오피에도 베트남년들 태국년들있고 대부분 국걸로 입국해서 화재들 뒷통수치고 가출할거겠지만 ㅋㅋㅋㅋㅋ']
sample_list += ['아 미친 존나 배고파', '교수 과제 왜 이따구로 냄?', 'ㅅㅂ 집가고 싶다']
sample_list += ['ㅈㄴ 게이같노 ㅋㅋ', '지랄하지말고 묻는 말에 대답이나해 이기야', '저 미친 ㄴ ㅋㅋㅋㅋㅋ', '한남 수준 실화노 ㅋㅋ']
sample_list += ['한남 수준 ㅉㅉ', '말하는 꼬라지만봐도 홍어 냄새 나네 라도냐?', '개슬람 다 불태워버리고 싶다', '시발 ㅈ같네']

In [ ]:
for sample in sample_list:
    input_text = f"""
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.\n\n
    Input(순화할 문장): {sample}\n
    Output(순화한 문장):
    """
    output_text = infer(input_text, model, tokenizer)
    # print(f"Input: {input_text}")
    print(f"Output: {output_text}")

Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): 에휴 씨발 물가 좆같이 올라서 먹을 수 있는게 없노

    Output(순화한 문장):
     물가 올라서 먹을 수 있는 게 없더라고
Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): 조선족 개새끼들 한국땅에서 못살게 존나 다 내쫓아야한다이기야

    Output(순화한 문장):
    ᄏᄏ조선족도 못 살게 됨 
Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): 개쌉레알이노 ᄏᄏᄏᄏ 웹툰보이가 그린웹툰들 몰입력 ᄊᄒᄐᄎ에 내용 ᄊᄒᄐᄎ 반면 웹툰작가꺼는 나 ᄊᄉᄐᄎ에 잼임

    Output(순화한 문장):
    ᄏᄏ 웹툰이 그린 웹툰들 밀면 좋더라 
Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): ᄏᄏᄏᄏ 상상속에서 맘충이랑 싸움했ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ일베에서나 키보드로 욕이나 하지 현실에선 말한마디나 제대로 하노?

    Output(순화한 문장):
    ᄏᄏ 상상속에서  싸움다녔다 ᄏᄏ 
Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): 요즘 유흥에도 베트남년들 개많더라 오피에도 베트남년들 태국년들있고 대부분 국걸로 입국해서 화재들 뒷통수치고 가출할거겠지만 ᄏᄏᄏᄏᄏ

    Output(순화한 문장):
    ᄏᄏ 요즘밍에도 베트남이랑 중국  많이 가는 거 같아 ᄏᄏ
Output: 
    Input 문장의 욕설, 혐오 표현을 순화한 응답을 작성하세요.


    Input(순화할 문장): 아 미친 존나 배고파

    Output(순화한 문장):
    헉 진